In [1]:
USE {
    dependencies {
        implementation("org.springframework.boot", "spring-boot-starter-data-jdbc", "3.5.6")
        implementation("org.postgresql:postgresql:42.7.7")
    }
}

In [2]:
%use dataframe

In [3]:
val USER = System.getenv("DB_USER")
val PASS = System.getenv("DB_PASS")
val URL = System.getenv("DB_URL")

println("$USER@$URL")
//println("$PASS")

val dbConfig = DbConnectionConfig(URL, USER, PASS)

val assetName = "QXMR"
val issuer = "QXMRTKAIIGLUREPIQPCMHCKWSIPDTUYFCFNYXQLTECSUJVYEMMDELBMDOEYB"

val query = """
select t.tick_time, tx.hash as transaction_hash, taker.identity as taker, maker.identity as maker, t.bid, t.price, t.number_of_shares
    from trades t
        join transactions tx on t.transaction_id = tx.id
        join entities taker on tx.source_entity_id = taker.id
        join entities maker on t.maker_id = maker.id
        join assets a on t.asset_id = a.id
    where a.issuer = '${issuer}' and a.name = '${assetName}'
    order by tick_time desc, t.id desc
"""

val df = DataFrame.readSqlQuery(dbConfig, query)
df.writeCsv("qxmr-trades.csv")


qx@jdbc:postgresql://localhost:5432/qx
